In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests

from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a set of 2000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs

In [3]:
# Add latitudes and longitudes to a list
coordinates = list(lats_lngs)
coordinates

[(12.20482875616321, -173.7047673831749),
 (37.47751336421835, 174.9601767164803),
 (17.2652421480404, 152.03352784617107),
 (45.66658968592412, -101.1509677020888),
 (-54.94926998764949, -163.4841600397742),
 (-19.341839275563473, 154.04611704618787),
 (-14.398423751561893, -55.32262844833697),
 (31.589341559909187, 2.346061091082362),
 (-10.50990619805907, -83.47740306598457),
 (-83.51010743626757, -138.57471516347331),
 (31.436095547294258, -95.20206161752795),
 (-17.179024251403703, -115.7834915926936),
 (-3.102923866925522, 75.60664750999698),
 (-31.723720178213576, -57.871981721899715),
 (13.933585884160038, 61.27538079950028),
 (39.777879046184864, -4.658216505009733),
 (-63.90391194588321, 164.11991448848823),
 (15.518964551381686, -23.26272433120681),
 (-5.815861435641651, 163.11787600152093),
 (29.274683884754467, -49.3739344273086),
 (-50.16045582683167, -175.13876915289242),
 (19.622634476337353, 130.7867265965363),
 (39.23365377164586, -127.68694950364704),
 (60.6577613627

In [4]:
# Get the nearest city using the citipy module
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique then add it to the cities list
    if city not in cities:
        cities.append(city)

# Check count to confirm sufficient cities
len(cities)

767

In [5]:
# Build URL for Weather API call using my weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# Test if URL works
test_city_url = url + "&q=" + "Boston"
test_city_weather = requests.get(test_city_url)
test_city_weather

<Response [200]>

In [7]:
# Get the JSON text of the "get" request
test_city_weather.json()

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 52.81,
  'feels_like': 50.18,
  'temp_min': 45.48,
  'temp_max': 59.38,
  'pressure': 1020,
  'humidity': 51},
 'visibility': 10000,
 'wind': {'speed': 5.01, 'deg': 0, 'gust': 0},
 'clouds': {'all': 40},
 'dt': 1651360430,
 'sys': {'type': 2,
  'id': 2009921,
  'country': 'US',
  'sunrise': 1651311643,
  'sunset': 1651362119},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [8]:
# Create an empty list to hold the weather data
weather_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the cities list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the URL, record, and set numbers and the city
    print(f"Processing record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append city weather information into weather_data list
        weather_data.append({"City": city.title(), 
                             "Country": city_country,
                             "Date": city_date,
                             "Lat": city_lat,
                             "Lng": city_lng,
                             "Max Temp": city_max_temp,
                             "Humidity": city_humidity,
                             "Cloudiness": city_clouds,
                             "Wind Speed": city_wind,
                             "Current Description": city_weather_description})        

# If an error is experience, skip the city
    except:
        print("City not found. Skipping...")
        pass       
        
# Indicate that Data Retrieval is complete.
print("--------------------------------")
print("Data Retrieval Complete.        ")
print("--------------------------------")

Beginning Data Retrieval     
-----------------------------
Processing record 1 of Set 1 | kapaa
Processing record 2 of Set 1 | nikolskoye
Processing record 3 of Set 1 | lorengau
Processing record 4 of Set 1 | mandan
Processing record 5 of Set 1 | avarua
Processing record 6 of Set 1 | yeppoon
Processing record 7 of Set 1 | nobres
Processing record 8 of Set 1 | aflu
City not found. Skipping...
Processing record 9 of Set 1 | chicama
Processing record 10 of Set 1 | rikitea
Processing record 11 of Set 1 | lufkin
Processing record 12 of Set 1 | hithadhoo
Processing record 13 of Set 1 | salto
Processing record 14 of Set 1 | salalah
Processing record 15 of Set 1 | talavera de la reina
Processing record 16 of Set 1 | bluff
Processing record 17 of Set 1 | vila do maio
Processing record 18 of Set 1 | auki
Processing record 19 of Set 1 | saint george
Processing record 20 of Set 1 | vaini
Processing record 21 of Set 1 | itoman
Processing record 22 of Set 1 | fortuna
Processing record 23 of Set 1 |

Processing record 38 of Set 4 | mendi
Processing record 39 of Set 4 | kahului
Processing record 40 of Set 4 | kodiak
Processing record 41 of Set 4 | varhaug
Processing record 42 of Set 4 | port blair
Processing record 43 of Set 4 | half moon bay
Processing record 44 of Set 4 | taoudenni
Processing record 45 of Set 4 | san patricio
Processing record 46 of Set 4 | gimli
Processing record 47 of Set 4 | makkaveyevo
Processing record 48 of Set 4 | mangrol
Processing record 49 of Set 4 | brigantine
Processing record 50 of Set 4 | adrar
Processing record 1 of Set 5 | qeshm
Processing record 2 of Set 5 | cherskiy
Processing record 3 of Set 5 | tombouctou
Processing record 4 of Set 5 | bengkulu
Processing record 5 of Set 5 | katsuura
Processing record 6 of Set 5 | kruisfontein
Processing record 7 of Set 5 | galgani
City not found. Skipping...
Processing record 8 of Set 5 | pathein
Processing record 9 of Set 5 | villa vazquez
City not found. Skipping...
Processing record 10 of Set 5 | saskylakh


Processing record 23 of Set 8 | opuwo
Processing record 24 of Set 8 | vicuna
Processing record 25 of Set 8 | lingao
Processing record 26 of Set 8 | suntar
Processing record 27 of Set 8 | melito di porto salvo
Processing record 28 of Set 8 | newcastle
Processing record 29 of Set 8 | atbasar
Processing record 30 of Set 8 | hailar
Processing record 31 of Set 8 | grindavik
Processing record 32 of Set 8 | kieta
Processing record 33 of Set 8 | emerald
Processing record 34 of Set 8 | huancabamba
Processing record 35 of Set 8 | maymanak
City not found. Skipping...
Processing record 36 of Set 8 | sept-iles
Processing record 37 of Set 8 | kalmunai
Processing record 38 of Set 8 | ladario
Processing record 39 of Set 8 | amderma
City not found. Skipping...
Processing record 40 of Set 8 | tumannyy
City not found. Skipping...
Processing record 41 of Set 8 | bukama
Processing record 42 of Set 8 | ekibastuz
Processing record 43 of Set 8 | bartow
Processing record 44 of Set 8 | baykit
Processing record 

Processing record 12 of Set 12 | ugoofaaru
Processing record 13 of Set 12 | nadym
Processing record 14 of Set 12 | pangnirtung
Processing record 15 of Set 12 | petropavlovsk-kamchatskiy
Processing record 16 of Set 12 | road town
Processing record 17 of Set 12 | acarau
Processing record 18 of Set 12 | saint anthony
Processing record 19 of Set 12 | gorontalo
Processing record 20 of Set 12 | sao miguel do oeste
City not found. Skipping...
Processing record 21 of Set 12 | kupang
Processing record 22 of Set 12 | paranaiba
Processing record 23 of Set 12 | praia da vitoria
Processing record 24 of Set 12 | sorong
Processing record 25 of Set 12 | port lincoln
Processing record 26 of Set 12 | alto vera
City not found. Skipping...
Processing record 27 of Set 12 | dourbali
Processing record 28 of Set 12 | cherepovets
Processing record 29 of Set 12 | tucumcari
Processing record 30 of Set 12 | collie
Processing record 31 of Set 12 | sinnamary
Processing record 32 of Set 12 | the valley
Processing re

Processing record 48 of Set 15 | marcona
City not found. Skipping...
Processing record 49 of Set 15 | kendari
Processing record 50 of Set 15 | saint combs
City not found. Skipping...
Processing record 1 of Set 16 | chagda
City not found. Skipping...
Processing record 2 of Set 16 | ampanihy
Processing record 3 of Set 16 | mahanoro
Processing record 4 of Set 16 | barbar
City not found. Skipping...
Processing record 5 of Set 16 | walsrode
Processing record 6 of Set 16 | terrace bay
Processing record 7 of Set 16 | cheremushskiy
Processing record 8 of Set 16 | pobe
Processing record 9 of Set 16 | ust-kulom
Processing record 10 of Set 16 | yima
Processing record 11 of Set 16 | edmundston
Processing record 12 of Set 16 | siocon
Processing record 13 of Set 16 | manzil tamim
City not found. Skipping...
Processing record 14 of Set 16 | zhitikara
City not found. Skipping...
Processing record 15 of Set 16 | calamar
Processing record 16 of Set 16 | port macquarie
Processing record 17 of Set 16 | ga

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame
weather_data_df = pd.DataFrame(weather_data)
weather_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,2022-04-30 23:18:32,22.0752,-159.3190,82.38,76,40,18.41,scattered clouds
1,Nikolskoye,RU,2022-04-30 23:18:33,59.7035,30.7861,37.80,78,96,7.14,overcast clouds
2,Lorengau,PG,2022-04-30 23:18:33,-2.0226,147.2712,82.94,74,100,1.32,overcast clouds
3,Mandan,US,2022-04-30 23:18:33,46.8267,-100.8896,52.38,59,100,29.93,overcast clouds
4,Avarua,CK,2022-04-30 23:18:34,-21.2078,-159.7750,84.25,74,20,9.22,few clouds
5,Yeppoon,AU,2022-04-30 23:18:34,-23.1333,150.7333,74.98,83,89,10.98,overcast clouds
6,Nobres,BR,2022-04-30 23:18:35,-14.7203,-56.3275,81.88,47,46,2.44,scattered clouds
7,Chicama,PE,2022-04-30 23:18:35,-7.8447,-79.1469,63.14,75,46,8.23,scattered clouds
8,Rikitea,PF,2022-04-30 23:18:36,-23.1203,-134.9692,75.97,75,68,10.67,moderate rain
9,Lufkin,US,2022-04-30 23:18:36,31.3382,-94.7291,85.48,65,75,3.44,broken clouds


In [10]:
# Create the output file (CSV)
output_data_file = "Weather_Database/WeatherPy_challenge.csv"

# Export the weather_data DataFrame into a CSV file
weather_data_df.to_csv(output_data_file, index_label="City_ID")